# Data Ingestion process

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MONIKA\\Deep_Learning_Projects\\NLP_Projects\\2_Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\MONIKA\\Deep_Learning_Projects\\NLP_Projects\\2_Text_Summarizer'

### Creating a custom data type using Dataclass package

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Here, 'DataIngestionConfig' class is not an actual Python class but a dataclass's class.

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import urllib.request as request
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_files(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """       
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

            

In [10]:
print(CONFIG_FILE_PATH)

config\config.yaml


In [11]:
print(CONFIG_FILE_PATH.as_posix())

config/config.yaml


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e

[2025-10-25 18:16:03,125: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-25 18:16:03,144: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-25 18:16:03,147: INFO: common: Created directory at: artifacts]
[2025-10-25 18:16:03,149: INFO: common: Created directory at: artifacts/data_ingestion]


[2025-10-25 18:16:07,838: INFO: 2766942487: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3570:D1063:F265E:23B64D:68FCC68A
Accept-Ranges: bytes
Date: Sat, 25 Oct 2025 12:46:04 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210053-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1761396364.703345,VS0,VE654
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a8de67d40f6ba0f255fde1ea7ebe17404108a6e0
Expires: Sat, 25 Oct 2025 12:51:04 GMT
Source-Age: 0

]
